<a href="https://colab.research.google.com/github/arminabdeh/Localization3D/blob/smlm/optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @markdown **Step 0** Prepare Colab instance (~5 mins)
# @markdown  1. install __luenn__ package
# @markdown  2. install __decode__ package
# @markdown  3. install __torchvision__
# @markdown  4. install __torch 1.12.0__
# @markdown  5. install __optuna__ and __optuna-dashboard__
# @markdown  3. install __torachtext__ and __torchaudio__

%%capture
!pip install Optuna
!pip install Optuna-dashboard
!pip uninstall -y torch torchvision torchaudio torchtext
!pip install torch==1.12.0
import torch
import yaml
!rm -r sample_data/*
!wget -c https://raw.githubusercontent.com/arminabdeh/Localization3D/smlm/gateway.yaml
with open('gateway.yaml') as f:
  gateway = yaml.safe_load(f)


wheel_spline = gateway['wheels']['spline_py310']
wheel_spline_name = "spline-0.11.1.dev0-cp310-cp310-linux_x86_64.whl"
!wget -O $wheel_spline_name $wheel_spline
!pip install $wheel_spline_name

wheel_decode = gateway['wheels']['decode']
!pip install $wheel_decode

wheel_luenn =  gateway['wheels']['luenn']
!pip install $wheel_luenn

In [2]:
#@markdown **Step 3** Mount Google Drive
#@markdown
#@markdown Execute this cell to connect your Google Drive
#@markdown This is important to save your trained model. An alternative would be to download it manually, this is however not recommended.


#@markdown * Click on the URL.

#@markdown * Sign in your Google Account.

#@markdown * Copy the authorization code.

#@markdown * Enter the authorization code.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

#@markdown Your Google Drive is accessible in Colab as `gdrive/My Drive/[YOUR Folder / File]`. **Important** It might take a while until files are being synced (particularly large files).
# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
from luenn.model import UNet
import decode.utils.param_io as param_io
from luenn.generic import fly_simulator
from torch.utils.data import Dataset, TensorDataset,DataLoader
import optuna
from tqdm import tqdm
from tqdm.notebook import tqdm

In [77]:
# @title Default title text
#@markdown > Set main parameters:
epochs = 100 # @param {type:"integer"}
training_size = 10 # @param {type:"integer"}
test_size = 10 # @param {type:"integer"}
#@markdown >> Set trail range of learning rate
lr_lim_down = 1e-5 # @param {type:"raw"}
lr_lim_up = 1e-3 # @param {type:"raw"}
#@markdown >> Set trail range of batch size
batch_size_min = 16 # @param {type:"integer"}
batch_size_max = 32 # @param {type:"integer"}

param_path = "/content/gdrive/MyDrive/optuna/param/param.yaml"
param = param_io.load_params(param_path)
calib_path = "/content/gdrive/MyDrive/optuna/calib/spline_calibration_3d_as_3dcal.mat"
param.InOut.calibration_file = calib_path
param.HyperParameter.pseudo_ds_size = training_size
param.TestSet.test_size = test_size
simulator = fly_simulator(param,report=False)


def objective(trial):
  model = UNet()
  model.to('cuda')
  #trials
  # optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "RMSprop", "SGD"])
  optimizer_name = 'Adam'
  lr = trial.suggest_float("lr", lr_lim_down, lr_lim_up, log=True)
  optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
  step = trial.suggest_int("step", 1, 20, log=True)
  # step = 10
  gamma = trial.suggest_float("gamma", 0.1, 1, log=True)
  # gamma = .5
  batch_size = trial.suggest_int("batch_size", batch_size_min, batch_size_max, log=True)

  #training
  model.train()


  criterion = torch.nn.MSELoss()
  lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step, gamma=gamma)

  tqdm_enum_train = tqdm(total=epochs, desc="Processing", unit=" iterations")

  for epoch in range(epochs):
    x_test, y_test, gt_test = simulator.ds_test()
    dataset_test = torch.utils.data.TensorDataset(x_test, y_test)
    dataloader_test  = DataLoader(dataset_test, batch_size=32,num_workers=0, shuffle=False, pin_memory=False)
    x_train, y_train, gt_train = simulator.ds_train()
    dataset_train = torch.utils.data.TensorDataset(x_train, y_train)
    dataloader_train  = DataLoader(dataset_train, batch_size=batch_size,num_workers=0,shuffle=False, pin_memory=False)
    tr_loss = 0
    steps_train = len(dataloader_train)
    steps_test = len(dataloader_test)
    # tqdm_enum_test = tqdm(total=steps_test, desc="Processing", unit=" iterations")
    for data in dataloader_train:
      inputs, labels = data
      inputs = inputs.cuda()
      labels = labels.cuda()
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      tr_loss += loss
      loss.backward()
      optimizer.step()
      inputs.cpu()
      labels.cpu()
      torch.cuda.empty_cache()
    tqdm_enum_train.update(1)
    # tqdm_enum_train.close()
    lr_scheduler.step()
    # validation
    val_loss = 0
    model.eval()
    with torch.no_grad():
      for data in dataloader_test:
        inputs, labels = data
        outputs = model(inputs.cuda())
        loss = criterion(outputs, labels.cuda())
        val_loss += loss.item()
        inputs.cpu()
        labels.cpu()
        torch.cuda.empty_cache()
        # tqdm_enum_test.update(1)
  # trial.report(val_loss,epoch)
  # tqdm_enum_test.close()
  val_loss = val_loss / steps_test
  if trial.should_prune():
    raise optuna.exceptions.TrialPruned()
  return val_loss


In [78]:
study = optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=200, timeout=60)

[I 2023-11-17 23:38:27,727] A new study created in memory with name: no-name-3fd278f9-0ca0-4e88-8834-02844470fe48


Processing:   0%|          | 0/100 [00:00<?, ? iterations/s]

[I 2023-11-17 23:39:43,253] Trial 0 finished with value: 86689.46875 and parameters: {'lr': 0.00012245039015301848, 'step': 20, 'gamma': 0.8169825482536909, 'batch_size': 16}. Best is trial 0 with value: 86689.46875.


In [56]:
print(study.best_trial)
print(study.best_params)

FrozenTrial(number=0, state=TrialState.COMPLETE, values=[102597.94618055556], datetime_start=datetime.datetime(2023, 11, 17, 23, 26, 48, 348775), datetime_complete=datetime.datetime(2023, 11, 17, 23, 27, 41, 565928), params={'optimizer': 'Adam', 'lr': 0.0009698758437767134, 'step': 2, 'gamma': 0.7789673493233854, 'batch_size': 25}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'optimizer': CategoricalDistribution(choices=('Adam', 'AdamW', 'RMSprop', 'SGD')), 'lr': FloatDistribution(high=0.001, log=True, low=1e-05, step=None), 'step': IntDistribution(high=20, log=True, low=1, step=1), 'gamma': FloatDistribution(high=1.0, log=True, low=0.1, step=None), 'batch_size': IntDistribution(high=32, log=True, low=16, step=1)}, trial_id=0, value=None)
{'optimizer': 'Adam', 'lr': 0.0009698758437767134, 'step': 2, 'gamma': 0.7789673493233854, 'batch_size': 25}
